## Imports

In [ ]:
!pip install ndlib
!pip install cdlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.0 MB/s eta 0:00:00
  Created wheel for python-igraph: filename=python_igraph-0.10.4-py3-none-any.whl size=9073 sha256=1a496c34cfc0c63d8a6b67b0f7a0541b03cdbd90b7fdc5f88f7533ddd6a81cd6
  Stored in directory: /root/.cache/pip/wheels/49/27/15/dcf90953e3e3322e6f3e447514b20cf39b53c6518cb4a7bace
Successfully built python-igraph
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.6/228.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━

In [ ]:
import networkx as nx
from igraph import *
import community
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import matplotlib.pyplot as plt
import cdlib
from cdlib import algorithms, NodeClustering
from cdlib import evaluation
import numpy as np
from networkx.classes.function import common_neighbors
from cdlib import evaluation, algorithms
from cdlib.utils import convert_graph_formats, __from_nx_to_graph_tool, affiliations2nodesets, nx_node_integer_mapping
import community as louvain_modularity
from networkx.generators.community import LFR_benchmark_graph
from collections import defaultdict
import math
from math import comb
import random
from networkx.algorithms.components.connected import connected_components
import time
import pandas as pd
import json
import csv
import copy
import operator as op
from functools import reduce

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'karateclub', 'leidenalg', 'infomap', 'graph_tool'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'karateclub', 'ASLPAw'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg'}


## Quality matrix

In [ ]:
def community_coverage(G, comm):
    # Community coverage should be more
    nodes_in_com = 0
    for n in G.nodes():
        for c in comm:
            if n in c and len(c)>=3:
                nodes_in_com += 1;
                break;
    
    return nodes_in_com/len(G.nodes())


def overlap_coverage(G, comm):
    total = 0
    for n in G.nodes():
        for c in comm:
            if n in c and len(c)>=3:
                total += 1;
    
    return total/len(G.nodes())


def extended_modularity(G, clusters):
    sum1 = 0
    n_comm = defaultdict(lambda: 0)  # {node: num of communities it is in}
    for comm in clusters:
        for node in comm:
            n_comm[node] = n_comm[node] + 1

    degree = G.degree()
    m2 = G.number_of_edges() * 2
    m = 0
    for U in clusters:
        i=0
        sum1=0
        while i < len(U):
            j = i + 1
            while j < len(U):
                x = (G.has_edge(U[i], U[j]) - ((G.degree(U[i]) * G.degree(U[j])) / (m2)))  / (
                            n_comm[U[i]] * n_comm[U[j]])
                sum1 = sum1 +  2*x
                j = j + 1
            i = i + 1
        m = m + sum1

    m = m / (m2)  # compute the final modularity
    return (m)


def modularity_overlap(G, comm):
    def ncr(n, r):
        r = min(r, n-r)
        numer = reduce(op.mul, range(n, n-r, -1), 1)
        denom = reduce(op.mul, range(1, r+1), 1)
        return numer // denom  

    n_comm = defaultdict(lambda: 0)  # {node: num of communities it is in}
    for c in comm:
        for node in c:
            n_comm[node] = n_comm[node] + 1

    mod_C = len(comm)
    num = 0
    for c in comm:
        nc = len(c)
        nec = 0
        for i in c:
            for j in c:
                if G.has_edge(i, j):
                    nec += 1

        num_num = 0
        for i in c:
            in_comm_connections = 0
            for j in c:
                if i != j and G.has_edge(i, j):
                    in_comm_connections += 1
            
            out_comm_connections = 0
            for j in G.nodes():
                if j not in c and j != i and G.has_edge(i, j):
                    out_comm_connections += 1
            
            num_num += ((in_comm_connections - out_comm_connections)/(G.degree(i)*n_comm[i]))
        
        num += ((num_num*nec)/(nc*ncr(nc, 2)))
            
    return num/mod_C


def overlapping_modularity(G, comm):
    N = G.number_of_edges() * 2
    res = comm # list of communities
    m=0
    for U in res:
        n=len(U);
        
        S=G.subgraph(U)
        
        rr=[]
        for kk in res:
            if not kk==U:
                rr.extend(kk)
        
        ov=list(set(U).intersection(set(rr)))
        
        sum1 = 0
        i = 0

        while i<len(U):
            j=i+1
            while j<len(U):
                if U[i] in ov:
                    o=S.degree(U[i])
                    o1=0
                    for ll in res:
                        if U[i] in ll:
                            S1=G.subgraph(ll)
                            o1=o1+S1.degree(U[i])
                    al1=o/o1
                else :   
                    al1=1

                if  U[j] in ov:
                    oo=S.degree(U[j])
                    oo1=0
                    for ll in res:
                        if U[j]in ll:
                            S1=G.subgraph(ll)
                            oo1=oo1+S1.degree(U[j]) 
                    al2=oo/oo1          
                else :
                    al2=1

                x = ((G.has_edge(U[j], U[i])-((G.degree(U[i])*G.degree(U[j]))/(2*N)))*al1*al2)
                sum1= sum1+2*x

                j=j+1
            i=i+1
        m=m+sum1

    m=m/(2*N) # compute the total modularity
    
    return (m)


def overlapping_permanence(G):
    mat = np.zeros(shape=(G.number_of_nodes(),G.number_of_nodes()))
    for i in range (0,G.number_of_nodes()):
        for j in range (0,G.number_of_nodes()):
            if(G.has_edge(i,j)):
                mat[i][j]=1  


    def intersection(lst1,lst2):
        temp=set(lst2)
        lst3=[value for value in lst1 if value in temp]    
        return lst3

    def calc(G,community,v):
        common=set(G[v])&set(community)
        res = 0
        for u in common:
            share = 0
            for c in cluster:
                if v in c and u in c:
                    share += 1
            res += 1 / share
        return res

    def calc2(G,community,v):
      res=set()
      for c in cluster:
          if v in c:
              res|=set(G[v])&set(c)
      return len(res)

    def internal_clustering_coefficient(G,cluster,v):
        nodes=intersection(cluster,nx.neighbors(G,v))
        nodes.append(v);
        H=G.subgraph(nodes)
        internal_CC=nx.clustering(H)
        b=calc(G,cluster,v)
        c=calc2(G,cluster,v)
        return internal_CC[v],b,c

    C_in = {}
    I = {}
    Iv= {}

    def cluster_update():
        global C_in
        global I
        global Iv
        C_in, I,Iv = {}, {},{}
        for i in cluster:
            for j in i:
                a, b, Iv[j] = internal_clustering_coefficient(G, i, j)
                if j not in C_in.keys():
                    C_in[j] = []
                if j not in I.keys():
                    I[j] = []
                C_in[j].append(a)
                I[j].append(b)          

    D=G.degree()
    def External_conn_max(G,cluster,v):
        res=0
        for i in cluster:
            if v not in i:
                res=max(res,len(set(G[v])&set(i)))
        return res

    E_max={}
    for i in G:
        E_max[i]=External_conn_max(G,cluster,i)
        
    # Permanence
    def permanence(E,D):
        perm={}
        cluster_update()
        for i in I:
            # print(i)
            n = len(I[i])
            # print(n)
            for j in range(0, n): 
                # print(i,I[i][j],E[i],D[i],C_in[i][j],Iv[i])
                if (i not in perm.keys()):
                    perm[i] = 0
                if(E_max[i]!=0):
                    m = 1 if Iv[i] == 0 else (I[i][j]/Iv[i])
          
                    temp =((I[i][j]/E[i])*(1/D[i]))-((1-C_in[i][j])*(m))
                    perm[i] += temp
                    #print("permanence", i, temp);
                elif(D[i]==0 and E_max[i]==0):
                    perm[i] +=0;
                else:
                    perm[i]+=1

        #print(perm[0])
        return perm

    perm=permanence(E_max,D)
    print("perm",perm)
    print("\n")
    print(perm.values()) 
    print("\n")
    print(np.sum(perm.values())) 
    perm_sum=(np.sum(perm.values()))
    perml=list(perm_sum)
    perma=np.mean(perml)
    print("1. Average Permanance: "+str(perma), "\n")

    #perm_avg= perm_sum/len(perm)
    #print("Average Permanance: "+str(perm_avg), "\n")
    print("Permanence of Nodes:")
    for i in perm:
      print ("Node", i , "=", perm[i])
      


## Accuracy Matrix

In [ ]:
def nf1(comm1, comm2):
    return evaluation.nf1(comm1, comm2)


def onmi(comm1, comm2):
    return evaluation.overlapping_normalized_mutual_information_LFK(comm1, comm2)


def nmi_max(comm1, comm2):
    return evaluation.overlapping_normalized_mutual_information_MGH(comm1, comm2)


## ERCNS
### Edge Reduction and Common Neighbor Selection

In [ ]:
def ERCNS(G, iter=0.2, neighbor_threshold=0.8):
    OG = copy.deepcopy(G)

    # plt.title('Original Graph', fontsize=40)
    # nx.draw(G, with_labels=True, node_color='r')
    # plt.show()
    deg=G.degree()
    n=len(deg)
    edges = G.number_of_edges()

    NUM_ITERATIONS = (int)(iter * edges)
    for i in range(0, NUM_ITERATIONS):
        edge_betweenness = nx.edge_betweenness_centrality(G).items()
        edge_to_delete = sorted(edge_betweenness, key=lambda pair: -pair[1])[0][0]
        G.remove_edge(*edge_to_delete)

    com=nx.connected_components(G)
    comm = []
    for x in com:
        comm.append(list(x))

    com = comm

    # count=1
    # for i in com :
    #     print(count, i)
    #     count+=1
    # plt.title('FINAL', fontsize=20)  
    # nx.draw(G, with_labels=True, node_color='r')
    # plt.show()

    # Conversion of disjoint communities to overlapping communities
    overlap_com = []
    threshold=neighbor_threshold
    for c in com:
        temp_list = []
        for n in c:
            temp_list.append(n)
            for nn in OG.neighbors(n):
                # If the neighbor is already community, then skip it
                if nn in c:
                    continue
                total_neighbor_count = len(list(OG.neighbors(n)))
                in_c_count = 0
                for nnn in OG.neighbors(n):
                    if nnn in c:
                        in_c_count += 1

                if in_c_count/total_neighbor_count >= threshold and nn not in temp_list:
                    temp_list.append(nn)
        overlap_com.append(temp_list)

    # print()
    # print("__________After overlapping__________")
    # count=1
    # for i in overlap_com :
    #     print(count, i)
    #     count+=1

    # print()
    # # Overlapping nodes:
    # for i, x in enumerate(overlap_com): 
    #     if i != (len(overlap_com)-1):
    #         overlap = set(overlap_com[i]).intersection(overlap_com[i+1])
    #         print(f"overlapping nodes between {i} and {i+1}: {overlap}")
    #         print(f"length: {len(overlap)}")

    return overlap_com

## SLPA

In [ ]:
# This is our implementation of SLPA
# def SLPA(G):
#     def node_importance(G, alpha):

#         # Compute the eigenvector centrality for the graph
#         ec = nx.eigenvector_centrality(G)

#         # Compute the local and global properties for each node
#         degrees = dict(G.degree())
#         clustering = nx.clustering(G)

#         # Compute the node importance using the given formula
#         node_imp = {}
#         for node in G.nodes():
#             node_imp[node] = alpha * ec[node] + (1 - alpha) * (degrees[node] + clustering[node])

#         return node_imp


#     def label_propagation(G, node_imp):

#         # Initialize each node with a unique label
#         labels = {node: i for i, node in enumerate(G.nodes())}

#         while True:
#             # Shuffle the nodes in a random order
#             nodes = list(G.nodes())
#             np.random.shuffle(nodes)

#             # Track whether any label has changed during this iteration
#             changed = False

#             # Propagate the labels to the neighbors of each node
#             for node in nodes:
#                 # Compute the frequencies of the labels among the neighbors
#                 freq = {}
#                 for neighbor in G.neighbors(node):
#                     if labels[neighbor] not in freq:
#                         freq[labels[neighbor]] = 0
#                     freq[labels[neighbor]] += node_imp[neighbor]

#                 # Assign the label with the highest frequency to the node
#                 max_label = labels[node]
#                 max_freq = freq.get(max_label, 0)
#                 for label, frequency in freq.items():
#                     if frequency > max_freq:
#                         max_label = label
#                         max_freq = frequency

#                 # Update the label of the node if it has changed
#                 if labels[node] != max_label:
#                     labels[node] = max_label
#                     changed = True

#             # Exit the loop if no label has changed during this iteration
#             if not changed:
#                 break

#         # Convert the labels into sets of nodes for each community
#         communities = []
#         for label in set(labels.values()):
#             community = [node for node in G.nodes() if labels[node] == label]
#             communities.append(community)

#         return communities

#     node_imp = node_importance(G, alpha=0.5)
#     communities = label_propagation(G, node_imp)

#     # overlap_com = []
#     # for com in communities:
#     #     print(list(com))
#     #     overlap_com = overlap_com.append(list(com))
#     return communities


def SLPA(G, t=21, r=0.1):
  coms = algorithms.slpa(G,  t, r)
  overlap_com = coms.communities

  return overlap_com

## LPAM

In [ ]:
def LPAM(G, k=2, threshold=0.4, distance = "amp"):
  coms = algorithms.lpam(G, k, threshold, distance)
  overlap_com = coms.communities

  return overlap_com

## Core Expansion

In [ ]:
def core_expansion(G):
  coms = algorithms.core_expansion(G)
  overlap_com = coms.communities

  return overlap_com

## Walkscan

In [ ]:
def walkscan(G):
  coms = algorithms.walkscan(G)
  overlap_com = coms.communities

  return overlap_com

## PERCOMVC

In [ ]:
def percomvc(G):
  coms = algorithms.percomvc(G)
  overlap_com = coms.communities

  return overlap_com

## UMSTMO

In [ ]:
def umstmo(G):
  coms = algorithms.umstmo(G)
  overlap_com = coms.communities

  return overlap_com

## LPANNI

In [ ]:
def LPANNI(G):
    coms = algorithms.lpanni(G)
    overlap_com = coms.communities

    return overlap_com

## OCDID

In [ ]:
def OCDID(G):  
    jaccard_similarity = {}
    clustering_coffecient = {}
    average_degree = {}
    average_similarity = {}
    contact_strength = {}
    information = {}
    max_degree = max(len(list(G.neighbors(u))) for u in G.nodes())
    # Information that can be propagated
    def f(I_u, I_v):
        if I_u-I_v < 0:
            return 0
        return math.exp(I_u-I_v)-1

    # Inititalization
    for v in G.nodes():
        jaccard_similarity[v] = {}
        contact_strength[v] = {}
    for v in G.nodes():
        neighbors_v = list(G.neighbors(v))
        no_of_neighbor_v = len(neighbors_v)
        # computing jaccaed similarity
        for u in neighbors_v :
            neighbors_u = list(G.neighbors(u))
            gamma_u = set(neighbors_u)
            gamma_v = set(neighbors_v)
            gamma_u.add(u)
            gamma_v.add(v)
            intersection = len(list(gamma_v.intersection(gamma_u)))
            union = len(gamma_u.union(gamma_v))
            jaccard_similarity[u][v] = float(intersection)/float(union)
        # computing contact strngth
        for u in neighbors_v:
            N_u = set(G.neighbors(u))
            N_v = set(G.neighbors(v))
            triangle_count = len(N_u.intersection(N_v))
            no_of_triangle = 0
            for node1 in neighbors_v:
                for node2 in neighbors_v:
                    if node1 == node2:
                        continue
                    if G.has_edge(node1, node2) == True:
                        no_of_triangle += 1
            no_of_triangle //= 2
            if no_of_triangle == 0:
                #contact_strength[v][u] = 0.0
                if G.degree(v) == 1:
                    contact_strength[v][u] = 1.0
                else:
                    contact_strength[v][u] = 0.0
            else:
                contact_strength[v][u] = float(triangle_count)/float(no_of_triangle)

        # clustering_coffecient = nx.clustering(G)
        # computing avarage neighbor degree
        total_degree = 0
        for u in G.neighbors(v):
            total_degree += len(list(G.neighbors(u)))
        average_degree[v] = float(total_degree)/(float(no_of_neighbor_v))
        #average_degree = nx.average_neighbor_degree(G)
        #computing average similarity
        total_similarity = 0
        for u in neighbors_v:
            total_similarity += len(set(G.neighbors(u)).intersection(neighbors_v))/len(set(G.neighbors(u)).union(neighbors_v))
        #total_similarity += jaccard_similarity[u][v]
        average_similarity[v] = float(total_similarity)/float(no_of_neighbor_v)
        # computing information
        degree_v = len(neighbors_v)
        information[v] = float(degree_v)/float(max_degree)
    
    Flag = True
    Threshold = 0.001
    while Flag:
        I_max = 0
        for v in G.nodes():
            information_shared = 0
            for u in G.neighbors(v):
                # Computing propagation value
                propagation_value = f(information[u], information[v])*jaccard_similarity[u][v]*contact_strength[v][u]
                # Computing information loss
                information_lost = f(information[u], information[v])*(1.0-jaccard_similarity[u][v])*average_similarity[v]/average_degree[v]
                if propagation_value < information_lost:
                    information_lost = propagation_value
                I_in = propagation_value-information_lost
                information_shared += I_in
                if I_max < I_in:
                    I_max = I_in
            information[v] += information_shared
        if I_max < Threshold:
            Flag = False

    #community_of_node detection
    community_of_node = {}
    community_number = 1
    Threshold = 0.001
    for v in G.nodes():
        if v not in community_of_node:
            for u in G.neighbors(v):
                if abs(information[v]-information[u]) < Threshold:
                    if v in community_of_node:
                        if u in community_of_node:
                            cc = community_of_node[u]
                            for node in community_of_node:
                                if community_of_node[node] == cc:
                                    community_of_node[node] = community_of_node[v]
                        else:
                            community_of_node[u] = community_of_node[v]
                    else:
                        if u in community_of_node:
                            community_of_node[v] = community_of_node[u]
                        else:
                            community_of_node[v] = community_number
                            community_of_node[u] = community_number
                            community_number += 1

    for node in G.nodes():
        if node not in community_of_node and G.degree(node) == 1:
            temp = list(G.neighbors(node))[0]
            if temp in community_of_node:
                community_of_node[node] = community_of_node[temp]

    communities = {}
    for node in community_of_node:
        if community_of_node[node] not in communities:
            communities[community_of_node[node]] = set()
        communities[community_of_node[node]].add(node)

    comm = []
    for i in communities:
        comm.append(list(communities[i]))

    overlapping_nodes = []
    for node in G.nodes():
        if node not in community_of_node:
            overlapping_nodes.append(node)
    return comm

## APAL

In [ ]:

def APAL(G, t=0.0005):
    graph = G
    communities = dict()
    community_count = 0    

    def evaluate(candidate_community, threshold):
        communities_to_remove = list()
        # if intraconnectivity greater than threshold than only we will proceed
        if fitness(candidate_community) < threshold:
            return
        selected_community = None
        temporary_max_value = 0
        for community in communities:
            # Jaccard Index
            temporary_value = len(communities[community].intersection(candidate_community)) / len(candidate_community.union(communities[community]))
            if candidate_community.issubset(communities[community]):
                return
            elif communities[community].issubset(candidate_community):
                communities_to_remove.append(community)
            elif temporary_value > threshold and temporary_value > temporary_max_value and fitness(candidate_community.union(communities[community])) >= threshold:
                temporary_max_value = temporary_value
                selected_community = community
        for community in communities_to_remove:
            communities.pop(community)
        if selected_community is not None:
            communities[selected_community] = candidate_community.union(communities[selected_community])
            return
        community_count = len(communities) + 1
        community_name = "comm" + str(community_count)
        communities[community_name] = candidate_community

    def fitness(candidate_community):  #for finding intraconnectivity
        sum_adjacent_vertices = 0
        for vertex in candidate_community:
            sum_adjacent_vertices += len(set(graph.neighbors(vertex)).intersection(set(candidate_community)))
        if sum_adjacent_vertices == 0:
            return -1
        community_order = len(candidate_community)
        # returning value of alpha for each generated candidate community
        return sum_adjacent_vertices / (community_order * (community_order - 1))

    def run_apal(t):
        for vertex in graph.nodes:
            # adjacent_vertices = graph.get_adjacency_list(vertex)
            adjacent_vertices = graph.neighbors(vertex) 
            for adjacent_vertex in adjacent_vertices:
                set1 = set(adjacent_vertices).difference({adjacent_vertex})
                set2 = set(graph.neighbors(adjacent_vertex)).difference({vertex})
                community_set = set1.intersection(set2)
                if len(community_set) != 0:
                    community_set.add(vertex)
                    community_set.add(adjacent_vertex)
                    evaluate(community_set, t)
        return [list(x) for x in communities.values()]
    
    return run_apal(t)


## Run on all algorithms

In [ ]:
path='./sample_data/'
filename='15rw_'
dataset='55' # dataset number
G = nx.Graph()   
OG = nx.Graph()
edge_file= open(path+filename+'t'+str(dataset)+'.csv','r')
edge_list=edge_file.readlines()
for edge in edge_list:
    edge=edge.split()
    G.add_node(int(edge[0]))
    G.add_node(int(edge[1]))
    G.add_edge(int(edge[0]), int(edge[1]))
    
    OG.add_node(int(edge[0]))
    OG.add_node(int(edge[1]))
    OG.add_edge(int(edge[0]), int(edge[1]))

run_accuracy = 0
if run_accuracy:
    gt_file =  open(path+filename+'comm_t'+str(dataset)+'.txt','r')
    gt_list=gt_file.readlines()
    gt_comm = []
    for gt_c in gt_list:
        gt_c_list = gt_c.split()
        gt_comm.append(list(map(int, gt_c_list)))
    gt_comm_node_cluster = NodeClustering(gt_comm, OG, "", overlap=True)

G=G.to_undirected()
OG=OG.to_undirected()

# print("-----------ERCNS-----------")
# comm = ERCNS(G, 0.05)
# print("------ Quality Metrics: ")
# print(f"Community Coverage: {community_coverage(OG, comm)}")
# # print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
# print(f"Extended Modularity: {extended_modularity(OG, comm)}")
# print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
# print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# # print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
# if run_accuracy:
#     print("------ Accuracy Metrics: ")
#     comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
#     print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
#     print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
#     print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------LPANNI-----------")
comm = LPANNI(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------SLPA-----------")
comm = SLPA(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------Core Expansion-----------")
comm = core_expansion(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------Walkscan-----------")
comm = walkscan(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------UMSTMO-----------")
comm = umstmo(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------OCDID-----------")
comm = OCDID(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------APAL-----------")
comm = APAL(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------LPAM-----------")
comm = LPAM(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")


print()
print("##############################################################")
print("-----------PERCOMVC-----------")
comm = percomvc(OG)
print("------ Quality Metrics: ")
print(f"Community Coverage: {community_coverage(OG, comm)}")
# print(f"Overlap Coverage: {overlap_coverage(OG, comm)}")
print(f"Extended Modularity: {extended_modularity(OG, comm)}")
print(f"Overlapping Modularity: {overlapping_modularity(OG, comm)}")
print(f"Modularity Overlap: {modularity_overlap(OG, comm)}")
# print(f"Overlapping Permanence: {overlapping_permanence(OG)}")
if run_accuracy:
    print("------ Accuracy Metrics: ")
    comm_node_cluster = NodeClustering(comm, OG, "", overlap=True)
    print(f"NF1: {nf1(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"ONMI: {onmi(comm_node_cluster, gt_comm_node_cluster).score}")
    print(f"NMI Max: {nmi_max(comm_node_cluster, gt_comm_node_cluster).score}")



##############################################################
-----------LPANNI-----------
------ Quality Metrics: 
Community Coverage: 0.9935627630601634
Extended Modularity: 0.7965599861356064
Overlapping Modularity: 0.42992618418081885
Modularity Overlap: 0.4004468878821984

##############################################################
-----------SLPA-----------
------ Quality Metrics: 
Community Coverage: 1.0
Extended Modularity: 0.766274761134052
Overlapping Modularity: 0.43399571376550367
Modularity Overlap: 0.21030721184900386

##############################################################
-----------Core Expansion-----------
------ Quality Metrics: 
Community Coverage: 0.9606338202525377
Extended Modularity: 0.4863455457635493
Overlapping Modularity: 0.2921789175599105
Modularity Overlap: 0.1980297517102382

##############################################################
-----------Walkscan-----------
------ Quality Metrics: 
Community Coverage: 0.2978460014855162
Extended Mo

KeyboardInterrupt: ignored